- http://matthewrocklin.com/blog/work/2017/03/28/dask-xgboost
- https://examples.dask.org/machine-learning/xgboost.html#Learn-more

# setup

In [1]:
from pathlib import Path
import dask.dataframe as dd
import pandas as pd
import dask
print('dask.__version__', dask.__version__)
import xgboost
import datetime
def dtnow(): return datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')


SEED=101
HOME='/data/git/recsys20'
p_in=f'{HOME}/input'

cols=[
'text_tokens',
'hashtags',
'tweet_id',
'present_media',
'present_links',
'present_domains',
'tweet_type',
'language',
'timestamp',

'engaged_user_id',
'engaged_follower_count',
'engaged_following_count',
'engaged_is_verified',
'engaged_account_creation_time',

'engaging_user_id',
'engaging_follower_count',
'engaging_following_count',
'engaging_is_verified',
'engaging_account_creation_time',

'engagee_follows_engager',
'reply_engagement_timestamp',
'retweet_engagement_timestamp',
'retweet_with_comment_engagement_timestamp',
'like_engagement_timestamp',
]


cols_val = cols[:-4]

cols_tgt_tmstmp=[
    'retweet_engagement_timestamp',
    'reply_engagement_timestamp',
    'like_engagement_timestamp',
    'retweet_with_comment_engagement_timestamp',
]
cols_tgt=[o.split('_engagement_timestamp')[0] for o in cols_tgt_tmstmp]
print(cols_tgt)

dask.__version__ 2.12.0
['retweet', 'reply', 'like', 'retweet_with_comment']


In [2]:
from sklearn.metrics import precision_recall_curve, auc, log_loss

def compute_prauc(pred, gt):
    prec, recall, thresh = precision_recall_curve(gt, pred)
    prauc = auc(recall, prec)
    return prauc

def calculate_ctr(gt):
    positive = len([x for x in gt if x == 1])
    ctr = positive/float(len(gt))
    return ctr

def compute_rce(pred, gt):
    cross_entropy = log_loss(gt, pred)
    data_ctr = calculate_ctr(gt)
    strawman_cross_entropy = log_loss(gt, [data_ctr for _ in range(len(gt))])
    return (1.0 - cross_entropy/strawman_cross_entropy)*100.0

# prepare data

In [3]:
# (rcss20) ➜  recsys20 git:(master) ✗ wc -l input/trn.tsv
#  148,075,238 data/trn.tsv
# (rcss20) ➜  recsys20 git:(master) ✗ wc -l input/val.tsv
#  15,127,684 data/val.tsv


In [4]:
trntmstmp=1584412344
datetime.datetime.fromtimestamp(trntmstmp).strftime('%Y-%m-%d %H:%M:%S')

'2020-03-17 02:32:24'

In [5]:
valtmstmp=1584405047
datetime.datetime.fromtimestamp(valtmstmp).strftime('%Y-%m-%d %H:%M:%S')

'2020-03-17 00:30:47'

In [7]:
ls $p_in

downloads.ipynb*        trn_1584412344_1e7.tsv*  val_1584405047_1e3.tsv
tmp.ipynb*              trn_1584412344_5e7.tsv   val_1584405047.tsv*
trn_1584412344_1e3.tsv  trn_1584412344.tsv*      val1e4.tsv*
trn_1584412344_1e4.tsv  trn1e5.tsv*


In [ ]:
dftrn=dd.read_csv(
    f'{p_in}/trn_{trntmstmp}.tsv',
#     f'{p_in}/trn_{trntmstmp}_1e4.tsv',
    sep='\x01', header=None, names=cols,)

In [15]:
dfval=dd.read_csv(
    f'{p_in}/val_{valtmstmp}.tsv',
#     f'{p_in}/val_{valtmstmp}_1e3.tsv',
    sep='\x01',header=None,names=cols_val,)

In [21]:
dfval.head()

,text_tokens,hashtags,tweet_id,present_media,present_links,present_domains,tweet_type,language,timestamp,engaged_user_id,engaged_follower_count,engaged_following_count,engaged_is_verified,engaged_account_creation_time,engaging_user_id,engaging_follower_count,engaging_following_count,engaging_is_verified,engaging_account_creation_time,engagee_follows_engager
0,101\t47185\t10157\t100986\t10343\t55422\t119\t...,NaN,7647B4E9DAF4C1D8973397DC2A04F3E3,Photo,NaN,NaN,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581703126,8A9AB92B775C62C4AB60DF6773A01571,13941,1216,False,1448292186,0000006C3074607050F1339DDCB890BB,27448,600,False,1520948869,True
1,101\t6006\t5086\t1939\t7418\t3601\t6406\t1913\...,NaN,CCBFBA5AFE7EFC03102EA8D0F86C4208,Photo,NaN,NaN,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,1581736431,187AC59639DA9A6F32F7CD118EDD58F7,476439,1478,False,1254447722,00000776B07587ECA9717BFC301F2D6E,102,659,False,1478011810,False
2,101\t56898\t137\t44851\t10317\t11490\t10112\t1...,NaN,E18C2DCFC5AF20C650A0FD94598E69B7,Video,NaN,NaN,Retweet,ECED8A16BE2A5E8871FD55F4842F16B1,1582061925,82626B53CB2AD3B469E4AE06EAA9D930,367,702,False,1518708926,00000860E80C67D8C46CE57C64DE9444,230,189,False,1541013180,True
3,101\t13497\t10437\t94005\t11161\t73632\t11067\...,NaN,26DC813FDF8546B757BB9141099F119E,NaN,D58137F9D688C88435FD64FBAEA82B97,E91CDEC8DC7ABF30592FA024616FF970,TopLevel,ECED8A16BE2A5E8871FD55F4842F16B1,1582110043,7AFE06FF54898A1E9C716F539831849E,278,1229,False,1243548061,00000865A1538142CDA5936B07FE4311,65,165,False,1452599043,True
4,101\t24781\t10152\t42041\t38268\t10301\t10798\...,NaN,30A33055566AAC9EB18734C4EAD11FE1,NaN,AEF0CC9FA7B389B9A2ADF1331F00B65B,42DD9E2D4B2C0B0A71E909A6049EC2C2,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581860270,D240DACE38CA84965270C86D47D3BF40,24313527,121,True,1177506290,00000865A1538142CDA5936B07FE4311,64,164,False,1452599043,False


In [16]:
%%time
lang2cnt=dict(dfval.language.value_counts().compute())

/data/anaconda3/envs/rcss20/lib/python3.7/site-packages/dask/core.py:121: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*(_execute_task(a, cache) for a in args))


CPU times: user 2min 21s, sys: 22 s, total: 2min 43s
Wall time: 1min 24s


In [19]:
len(dfval)

/data/anaconda3/envs/rcss20/lib/python3.7/site-packages/dask/core.py:121: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*(_execute_task(a, cache) for a in args))


15127684

In [18]:
nval=len(dfval).compute

(Delayed('int-ad4c4931-3f8a-453c-abc9-31797d754d1c'), 20)

In [23]:
ttl=sum(lang2cnt.values())
lang2perc={k:v/ttl for k,v in lang2cnt.items()}
lang2perc

{'D3164C7FBCF2565DDF915B1B3AEFB1DC': 0.4281568150154379,
 '22C448FF81263D4BAF2A176145EE9EAD': 0.15653506511637869,
 '06D61DCBBE938971E1EA0C38BD9B5446': 0.08684409325313776,
 'ECED8A16BE2A5E8871FD55F4842F16B1': 0.07187590645071645,
 'B9175601E87101A984A50F8A62A1C374': 0.0510950651798385,
 '4DC22C3F31C5C43721E6B5815A595ED6': 0.03233621220538451,
 '167115458A0DBDFF7E9C0C53A83BAC9B': 0.02869156970756396,
 '125C57F4FA6D4E110983FB11B52EFD4E': 0.02251587222472389,
 '022EC308651FACB02794A8147AEE1B78': 0.022084940431066645,
 'FA3F382BC409C271E3D6EAF8BE4648DD': 0.020434654769361918,
 '9BF3403E0EB7EA8A256DA9019C0B0716': 0.01711266575901506,
 '975B38F44D65EE42A547283787FF5A21': 0.008900304897960587,
 '2996EB2FE8162C076D070A4C8D6532CD': 0.008491055207128863,
 'FF60A88F53E63000266F8B9149E35AD9': 0.005443463784674508,
 '717293301FE296B0B61950D041485825': 0.005351975887386331,
 '3E16B11B7ADE3A22DDFC4423FBCEAD5D': 0.005259694742433805,
 '3820C29CBCA409A33BADF68852057C4A': 0.005107391190878921,
 '9ECD42

In [24]:
len(lang2perc)

66